In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
import scipy.stats as st
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Read group stage schedule and teams' three letters' code dictionary and teams' offensse and defense strength




In [ ]:
spreadsheet = pd.read_excel('/content/drive/MyDrive/Sports_Analytics_Group_Project/Halfway_Simulation.xlsx',sheet_name='MATACHES & WINNING PROB')

In [ ]:
spreadsheet.head()

,eventID,TEAM1,TEAM2,TEAM1_xG,TEAM2_xG,TEAM1WIN_PROB,DRAW_PROB,TEAM2WIN_PROB,TEAM1_sim,TEAM2_sim,...,Offense Strength,Defense Strength,score,TEAM1.1,TEAM2.1,score.1,SIM1,SIM2,Unnamed: 20,rand()
0,1.0,QAT,SEN,0.557974,1.385113,0.149569,0.277303,0.573128,NaN,NaN,...,2.130013,0.345538,0.0,0.572368,0.250296,0.0,0.0,0.0,NaN,0.337280
1,2.0,NED,ECU,1.735667,0.909427,0.568772,0.235863,0.195365,NaN,NaN,...,1.547629,0.717944,1.0,0.891734,0.596983,1.0,NaN,NaN,NaN,0.953278
2,3.0,ECU,SEN,0.526775,1.203814,0.162236,0.308655,0.529109,NaN,NaN,...,1.746687,0.973624,2.0,0.980833,0.837084,2.0,NaN,NaN,NaN,0.815180
3,4.0,NED,QAT,1.997064,0.963289,0.613882,0.210954,0.175164,NaN,NaN,...,2.671386,0.238977,3.0,0.997404,0.947939,3.0,NaN,NaN,NaN,0.307108
4,5.0,WAL,IRN,0.920314,1.800492,0.190117,0.229311,0.580572,NaN,NaN,...,1.182788,0.965853,4.0,0.999716,0.986326,4.0,NaN,NaN,NaN,0.040400


In [ ]:
spreadsheet.columns

Index(['eventID', 'TEAM1', 'TEAM2', 'TEAM1_xG', 'TEAM2_xG', 'TEAM1WIN_PROB',
       'DRAW_PROB', 'TEAM2WIN_PROB', 'TEAM1_sim', 'TEAM2_sim', 'Unnamed: 10',
       'Team', 'Offense Strength', 'Defense Strength', 'score', 'TEAM1.1',
       'TEAM2.1', 'score.1', 'SIM1', 'SIM2', 'Unnamed: 20', 'rand()'],
      dtype='object')

# team rating

In [ ]:
team_rating = spreadsheet[['Team','Offense Strength','Defense Strength']]

In [ ]:
team_rating = team_rating.loc[:31]

In [ ]:
team_rating.head()

,Team,Offense Strength,Defense Strength
0,ARG,2.130013,0.345538
1,AUS,1.547629,0.717944
2,BEL,1.746687,0.973624
3,BRA,2.671386,0.238977
4,CAN,1.182788,0.965853


# group stage games 

In [ ]:
group_game = spreadsheet[['TEAM1', 'TEAM2', 'TEAM1_xG', 'TEAM2_xG', 'TEAM1WIN_PROB',
       'DRAW_PROB', 'TEAM2WIN_PROB', 'TEAM1_sim', 'TEAM2_sim']]

In [ ]:
group_game = group_game.loc[:47]

In [ ]:
group_game.head()

,TEAM1,TEAM2,TEAM1_xG,TEAM2_xG,TEAM1WIN_PROB,DRAW_PROB,TEAM2WIN_PROB,TEAM1_sim,TEAM2_sim
0,QAT,SEN,0.557974,1.385113,0.14947,0.277303,0.573128,1.0,3
1,NED,ECU,1.735667,0.909427,0.568323,0.235863,0.195365,3.0,1
2,ECU,SEN,0.526775,1.203814,0.162198,0.308655,0.529109,0.0,1
3,NED,QAT,1.997064,0.963289,0.612788,0.210954,0.175164,1.0,0
4,WAL,IRN,0.920314,1.800492,0.189549,0.229311,0.580572,1.0,3


In [ ]:
for index, row in group_game.iterrows():
    prob_draw =float()
    prob_t1_win =float()
    prob_t2_win =float()
    for x in range(0,8,1): #number of goals home team
        for y in range(0,8,1): #number of goals away team
            prob = st.poisson.pmf(x,row['TEAM1_xG']) * st.poisson.pmf(y,row['TEAM2_xG'])
            if x == y:
                prob_draw += prob
            elif x > y:
                prob_t1_win += prob
            else:
                prob_t2_win += prob
    group_game.at[index,'TEAM1WIN_PROB'] = prob_t1_win
    group_game.at[index,'DRAW_PROB'] = prob_draw
    group_game.at[index,'TEAM2WIN_PROB'] = prob_t2_win


In [ ]:
group_game.head()

,TEAM1,TEAM2,TEAM1_xG,TEAM2_xG,TEAM1WIN_PROB,DRAW_PROB,TEAM2WIN_PROB,TEAM1_sim,TEAM2_sim
0,QAT,SEN,0.557974,1.385113,0.14947,0.277303,0.573128,1.0,3
1,NED,ECU,1.735667,0.909427,0.568323,0.235863,0.195365,3.0,1
2,ECU,SEN,0.526775,1.203814,0.162198,0.308655,0.529109,0.0,1
3,NED,QAT,1.997064,0.963289,0.612788,0.210954,0.175164,1.0,0
4,WAL,IRN,0.920314,1.800492,0.189549,0.229311,0.580572,1.0,3


#Single match simulation

In [ ]:
def sim_group(group_game):
    for index, row in group_game.iterrows():
        group_game.at[index,'TEAM1_sim'] = int(np.random.poisson(row['TEAM1_xG'],1))
        group_game.at[index,'TEAM2_sim'] = int(np.random.poisson(row['TEAM2_xG'],1))
    return group_game

In [ ]:
group_game = sim_group(group_game)

# Team group stage rankings

In [ ]:
spreadsheet1 = pd.read_excel('/content/drive/MyDrive/Sports_Analytics_Group_Project/Halfway_Simulation.xlsx',sheet_name='TEAMS')

In [ ]:
spreadsheet1 = spreadsheet1.loc[:32]

# Teams survived in knockout stages

In [ ]:
stats = spreadsheet1.copy()

In [ ]:
stats[['top16','top8','top4','top2','Cham']] = 0

In [ ]:
stats.loc[stats.index[stats['Team Name'] == 'NED'].tolist(),'top16'] +=1

In [ ]:
stats.head()

,Team Name,Group,Full Name,top16,top8,top4,top2,Cham
0,ECU,A,Ecuador,0,0,0,0,0
1,NED,A,Netherlands,1,0,0,0,0
2,QAT,A,Qatar,0,0,0,0,0
3,SEN,A,Senegal,0,0,0,0,0
4,ENG,B,England,0,0,0,0,0


# World cup simulation

In [ ]:
###Single Trial Simulation!!!!
###
###
def sim_wc(num):
    spreadsheet1 = pd.read_excel('/content/drive/MyDrive/Sports_Analytics_Group_Project/Halfway_Simulation.xlsx',sheet_name='TEAMS')
    spreadsheet1 = spreadsheet1.loc[:32]
    stats = spreadsheet1.copy()
    stats[['top16','top8','top4','top2','Cham']] = 0

    group_stage =spreadsheet1

    spreadsheet = pd.read_excel('/content/drive/MyDrive/Sports_Analytics_Group_Project/Halfway_Simulation.xlsx',sheet_name='MATACHES & WINNING PROB')
    group_game = spreadsheet[['TEAM1', 'TEAM2', 'TEAM1_xG', 'TEAM2_xG', 'TEAM1WIN_PROB',
       'DRAW_PROB', 'TEAM2WIN_PROB', 'TEAM1_sim', 'TEAM2_sim']]
    group_game = group_game.loc[:47]

    team_rating = spreadsheet[['Team','Offense Strength','Defense Strength']]
    team_rating = team_rating.loc[:31]

    for index, row in group_game.iterrows():
        prob_draw =float()
        prob_t1_win =float()
        prob_t2_win =float()
        for x in range(0,8,1): #number of goals home team
            for y in range(0,8,1): #number of goals away team
                prob = st.poisson.pmf(x,row['TEAM1_xG']) * st.poisson.pmf(y,row['TEAM2_xG'])
                if x == y:
                    prob_draw += prob
                elif x > y:
                    prob_t1_win += prob
                else:
                    prob_t2_win += prob
        group_game.at[index,'TEAM1WIN_PROB'] = prob_t1_win
        group_game.at[index,'DRAW_PROB'] = prob_draw
        group_game.at[index,'TEAM2WIN_PROB'] = prob_t2_win
    

    for i in range(1,num):
        group_game = sim_group(group_game)
        group_stage["points"] = 0
        group_stage["total_goals"] = 0
        group_stage['total_goals_received'] = 0
        group_stage['goal_diff'] = 0
        for index, row in group_stage.iterrows():
            for index1,row1 in group_game.iterrows():
                if row1['TEAM1'] == row['Team Name']:
                  group_stage.at[index,'total_goals']+= row1['TEAM1_sim']
                  group_stage.at[index,'total_goals_received']+= row1['TEAM2_sim']
                  if row1['TEAM1_sim'] > row1['TEAM2_sim']:
                        group_stage.at[index,'points'] = group_stage.at[index,'points'] + 3
                  elif row1['TEAM1_sim'] == row1['TEAM2_sim']:
                        group_stage.at[index,'points'] = group_stage.at[index,'points'] + 1
                  else:
                        continue
                elif row1['TEAM2'] == row['Team Name']:
                  group_stage.at[index,'total_goals']+= row1['TEAM2_sim']
                  group_stage.at[index,'total_goals_received']+= row1['TEAM1_sim']
                  if row1['TEAM2_sim'] > row1['TEAM1_sim']:
                        group_stage.at[index,'points'] +=3
                  elif row1['TEAM1_sim'] == row1['TEAM2_sim']:
                        group_stage.at[index,'points'] +=1
                  else:
                        continue
                else:
                    continue
        group_stage['goal_diff'] = group_stage["total_goals"] - group_stage['total_goals_received']
        knockout_stage=[['T1','T2','TEAM1_xG','TEAM2_xG','TEAM1_sim','TEAM2_sim']]
        groupA = group_stage.loc[0:3].sort_values(by=['points', 'goal_diff'],ascending = False).reset_index(drop=True)
        groupB = group_stage.loc[4:7].sort_values(by=['points', 'goal_diff'],ascending = False).reset_index(drop=True)
        groupC = group_stage.loc[8:11].sort_values(by=['points', 'goal_diff'],ascending = False).reset_index(drop=True)
        groupD = group_stage.loc[12:15].sort_values(by=['points', 'goal_diff'],ascending = False).reset_index(drop=True)
        groupE = group_stage.loc[16:19].sort_values(by=['points', 'goal_diff'],ascending = False).reset_index(drop=True)
        groupF = group_stage.loc[20:23].sort_values(by=['points', 'goal_diff'],ascending = False).reset_index(drop=True)
        groupG = group_stage.loc[24:27].sort_values(by=['points', 'goal_diff'],ascending = False).reset_index(drop=True)
        groupH = group_stage.loc[28:31].sort_values(by=['points', 'goal_diff'],ascending = False).reset_index(drop=True)

        mat1 = pd.DataFrame([[groupA.loc[0]['Team Name'],groupB.loc[1]['Team Name'],0,0,0,0]],columns=['T1','T2','TEAM1_xG','TEAM2_xG','TEAM1_sim','TEAM2_sim'])
        mat2 = pd.DataFrame([[groupA.loc[1]['Team Name'],groupB.loc[0]['Team Name'],0,0,0,0]],columns=['T1','T2','TEAM1_xG','TEAM2_xG','TEAM1_sim','TEAM2_sim'])
        mat3 = pd.DataFrame([[groupC.loc[0]['Team Name'],groupD.loc[1]['Team Name'],0,0,0,0]],columns=['T1','T2','TEAM1_xG','TEAM2_xG','TEAM1_sim','TEAM2_sim'])
        mat4 = pd.DataFrame([[groupC.loc[1]['Team Name'],groupD.loc[0]['Team Name'],0,0,0,0]],columns=['T1','T2','TEAM1_xG','TEAM2_xG','TEAM1_sim','TEAM2_sim'])
        mat5 = pd.DataFrame([[groupE.loc[0]['Team Name'],groupF.loc[1]['Team Name'],0,0,0,0]],columns=['T1','T2','TEAM1_xG','TEAM2_xG','TEAM1_sim','TEAM2_sim'])
        mat6 = pd.DataFrame([[groupE.loc[1]['Team Name'],groupF.loc[0]['Team Name'],0,0,0,0]],columns=['T1','T2','TEAM1_xG','TEAM2_xG','TEAM1_sim','TEAM2_sim'])
        mat7 = pd.DataFrame([[groupG.loc[0]['Team Name'],groupH.loc[1]['Team Name'],0,0,0,0]],columns=['T1','T2','TEAM1_xG','TEAM2_xG','TEAM1_sim','TEAM2_sim'])
        mat8 = pd.DataFrame([[groupG.loc[1]['Team Name'],groupH.loc[0]['Team Name'],0,0,0,0]],columns=['T1','T2','TEAM1_xG','TEAM2_xG','TEAM1_sim','TEAM2_sim'])

        round16 = [mat1,mat2,mat3,mat4,mat5,mat6,mat7,mat8]
        
        for i in round16:
            stats.loc[stats.index[stats['Team Name'] == i.loc[0]['T1']].tolist(),'top16'] +=1
            stats.loc[stats.index[stats['Team Name'] == i.loc[0]['T2']].tolist(),'top16'] +=1
            for index, row in team_rating.iterrows():
                if row['Team'] == i.loc[0]['T1']:
                    os1 = row['Offense Strength']
                    ds1 = row['Defense Strength']
            for index, row in team_rating.iterrows():
                if row['Team'] == i.loc[0]['T2']:
                    os2 = row['Offense Strength']
                    ds2 = row['Defense Strength']
            i.at[0,['TEAM1_xG']] = os1 * ds2
            i.at[0,['TEAM2_xG']] = os2 * ds1
        for i in round16:
            sim_group(i)

        top8 = []
        for i in round16:
            if i.loc[0]['TEAM1_sim'] > i.loc[0]['TEAM2_sim']:
              top8.append(str(i.loc[0]['T1']))
            elif i.loc[0]['TEAM1_sim'] == i.loc[0]['TEAM2_sim']:
                c = np.random.rand()
                if i.loc[0]['TEAM1_xG'] > i.loc[0]['TEAM2_xG'] and c <=0.3:
                    top8.append(str(i.loc[0]['T1']))
                elif i.loc[0]['TEAM1_xG'] < i.loc[0]['TEAM2_xG'] and c <=0.3:
                    top8.append(str(i.loc[0]['T2']))
                elif i.loc[0]['TEAM1_xG'] > i.loc[0]['TEAM2_xG'] and c >0.3:
                    top8.append(str(i.loc[0]['T2']))
                else:
                    top8.append(str(i.loc[0]['T1']))
            else:
              top8.append(str(i.loc[0]['T2']))
        print(top8)

        mat21 = pd.DataFrame([[top8[0],top8[2],0,0,0,0]],columns=['T1','T2','TEAM1_xG','TEAM2_xG','TEAM1_sim','TEAM2_sim'])
        mat22 = pd.DataFrame([[top8[1],top8[3],0,0,0,0]],columns=['T1','T2','TEAM1_xG','TEAM2_xG','TEAM1_sim','TEAM2_sim'])
        mat23 = pd.DataFrame([[top8[4],top8[6],0,0,0,0]],columns=['T1','T2','TEAM1_xG','TEAM2_xG','TEAM1_sim','TEAM2_sim'])
        mat24 = pd.DataFrame([[top8[5],top8[7],0,0,0,0]],columns=['T1','T2','TEAM1_xG','TEAM2_xG','TEAM1_sim','TEAM2_sim'])

        round8 =[mat21,mat22,mat23,mat24]

        for i in round8:
            stats.loc[stats.index[stats['Team Name'] == i.loc[0]['T1']].tolist(),'top8'] +=1
            stats.loc[stats.index[stats['Team Name'] == i.loc[0]['T2']].tolist(),'top8'] +=1
            for index, row in team_rating.iterrows():
                if row['Team'] == i.loc[0]['T1']:
                    os1 = row['Offense Strength']
                    ds1 = row['Defense Strength']
            for index, row in team_rating.iterrows():
                if row['Team'] == i.loc[0]['T2']:
                    os2 = row['Offense Strength']
                    ds2 = row['Defense Strength']
            i.at[0,['TEAM1_xG']] = os1 * ds2
            i.at[0,['TEAM2_xG']] = os2 * ds1

        for i in round8:
            sim_group(i)

        top4 = []
        for i in round8:
            if i.loc[0]['TEAM1_sim'] > i.loc[0]['TEAM2_sim']:
              top4.append(str(i.loc[0]['T1']))
            elif i.loc[0]['TEAM1_sim'] == i.loc[0]['TEAM2_sim']:
                b = np.random.rand()
                if i.loc[0]['TEAM1_xG'] > i.loc[0]['TEAM2_xG'] and b <=0.3:
                    top4.append(str(i.loc[0]['T1']))
                elif i.loc[0]['TEAM1_xG'] < i.loc[0]['TEAM2_xG'] and b <=0.3:
                    top4.append(str(i.loc[0]['T2']))
                elif i.loc[0]['TEAM1_xG'] > i.loc[0]['TEAM2_xG'] and b >0.3:
                    top4.append(str(i.loc[0]['T2']))
                else:
                    top4.append(str(i.loc[0]['T1']))
            else:
              top4.append(str(i.loc[0]['T2']))
        print(top4)

        mat31 = pd.DataFrame([[top4[0],top4[2],0,0,0,0]],columns=['T1','T2','TEAM1_xG','TEAM2_xG','TEAM1_sim','TEAM2_sim'])
        mat32 = pd.DataFrame([[top4[1],top4[3],0,0,0,0]],columns=['T1','T2','TEAM1_xG','TEAM2_xG','TEAM1_sim','TEAM2_sim'])

        semi =[mat31,mat32]

        for i in semi:
            stats.loc[stats.index[stats['Team Name'] == i.loc[0]['T1']].tolist(),'top4'] +=1
            stats.loc[stats.index[stats['Team Name'] == i.loc[0]['T2']].tolist(),'top4'] +=1
            for index, row in team_rating.iterrows():
                if row['Team'] == i.loc[0]['T1']:
                    os1 = row['Offense Strength']
                    ds1 = row['Defense Strength']
            for index, row in team_rating.iterrows():
                if row['Team'] == i.loc[0]['T2']:
                    os2 = row['Offense Strength']
                    ds2 = row['Defense Strength']
            i.at[0,['TEAM1_xG']] = os1 * ds2
            i.at[0,['TEAM2_xG']] = os2 * ds1

        for i in semi:
            sim_group(i)

        top2 = []
        for i in semi:
            if i.loc[0]['TEAM1_sim'] > i.loc[0]['TEAM2_sim']:
              top2.append(str(i.loc[0]['T1']))
            elif i.loc[0]['TEAM1_sim'] == i.loc[0]['TEAM2_sim']:
                a = np.random.rand()
                if i.loc[0]['TEAM1_xG'] > i.loc[0]['TEAM2_xG'] and a <=0.3:
                    top2.append(str(i.loc[0]['T1']))
                elif i.loc[0]['TEAM1_xG'] < i.loc[0]['TEAM2_xG'] and a <=0.3:
                    top2.append(str(i.loc[0]['T2']))
                elif i.loc[0]['TEAM1_xG'] > i.loc[0]['TEAM2_xG'] and a >0.3:
                    top2.append(str(i.loc[0]['T2']))
                else:
                    top2.append(str(i.loc[0]['T1']))
            else:
              top2.append(str(i.loc[0]['T2']))
        print(top2)

        final = pd.DataFrame([[top2[0],top2[1],0,0,0,0]],columns=['T1','T2','TEAM1_xG','TEAM2_xG','TEAM1_sim','TEAM2_sim'])

        fin = [final]

        for i in fin:
            stats.loc[stats.index[stats['Team Name'] == i.loc[0]['T1']].tolist(),'top2'] +=1
            stats.loc[stats.index[stats['Team Name'] == i.loc[0]['T2']].tolist(),'top2'] +=1
            for index, row in team_rating.iterrows():
                if row['Team'] == i.loc[0]['T1']:
                    os1 = row['Offense Strength']
                    ds1 = row['Defense Strength']
            for index, row in team_rating.iterrows():
                if row['Team'] == i.loc[0]['T2']:
                    os2 = row['Offense Strength']
                    ds2 = row['Defense Strength']
            i.at[0,['TEAM1_xG']] = os1 * ds2
            i.at[0,['TEAM2_xG']] = os2 * ds1
        for i in fin:
            sim_group(i)
            print(i)
        top1 = []
        for i in fin:
            if i.loc[0]['TEAM1_sim'] > i.loc[0]['TEAM2_sim']:
              top1.append(str(i.loc[0]['T1']))
            elif i.loc[0]['TEAM1_sim'] == i.loc[0]['TEAM2_sim']:
                if i.loc[0]['TEAM1_xG'] > i.loc[0]['TEAM2_xG']:
                    top1.append(str(i.loc[0]['T1']))
                else:
                    top1.append(str(i.loc[0]['T2']))
            else:
              top1.append(str(i.loc[0]['T2']))
        stats.loc[stats.index[stats['Team Name'] == top1[0]].tolist(),'Cham'] +=1
        print(top1)
    print(stats)

In [ ]:
sim_wc(1000)

Streaming output truncated to the last 5000 lines.
['IRN', 'DEN', 'BRA', 'CMR']
['BRA', 'CMR']
    T1   T2  TEAM1_xG  TEAM2_xG  TEAM1_sim  TEAM2_sim
0  BRA  CMR  2.004859  0.394339          0          0
['BRA']
['IRN', 'ENG', 'FRA', 'SAA', 'BEL', 'MOR', 'POR', 'KOR']
['IRN', 'ENG', 'BEL', 'KOR']
['IRN', 'KOR']
    T1   T2  TEAM1_xG  TEAM2_xG  TEAM1_sim  TEAM2_sim
0  IRN  KOR  0.931904  1.030246          3          0
['IRN']
['SEN', 'STA', 'ARG', 'TUN', 'CRO', 'BEL', 'BRA', 'KOR']
['ARG', 'TUN', 'BRA', 'BEL']
['ARG', 'BEL']
    T1   T2  TEAM1_xG  TEAM2_xG  TEAM1_sim  TEAM2_sim
0  ARG  BEL  2.073833  0.603546          4          0
['ARG']
['NED', 'SEN', 'SAA', 'ARG', 'SP', 'CAN', 'BRA', 'KOR']
['SAA', 'SEN', 'BRA', 'CAN']
['BRA', 'CAN']
    T1   T2  TEAM1_xG  TEAM2_xG  TEAM1_sim  TEAM2_sim
0  BRA  CAN  2.580167  0.282659          4          0
['BRA']
['SEN', 'IRN', 'TUN', 'DEN', 'CRO', 'MOR', 'BRA', 'SER']
['TUN', 'IRN', 'BRA', 'MOR']
['BRA', 'IRN']
    T1   T2  TEAM1_xG  TEAM2_xG  TEAM1